### transform_img
This script can transform images by swirl effect.

Change input_dir & output_dir 

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
def swirl_effect(img, angle=180, strength=1.0, center_x_ratio=0.5, center_y_ratio=0.5):
    """
    旋轉扭曲效果：只在一定半徑內做 swirl，外圈保持原圖，避免邊界變成條紋。

    參數:
        img: 輸入圖片 (BGR格式)
        angle: 中心的最大旋轉角度（度數）
        strength: 扭曲強度，控制影響範圍 (0.0-2.0)
        center_x_ratio: 旋轉中心 X 位置比例 (0.0-1.0)
        center_y_ratio: 旋轉中心 Y 位置比例 (0.0-1.0)
    """
    height, width = img.shape[:2]

    # 旋轉中心
    center_x = width * center_x_ratio
    center_y = height * center_y_ratio

    # 最大半徑（影響範圍）
    max_radius = min(width, height) * strength / 2.0
    if max_radius <= 0:
        return img.copy()

    # 座標網格
    y, x = np.indices((height, width), dtype=np.float32)

    dx = x - center_x
    dy = y - center_y
    r = np.sqrt(dx**2 + dy**2)
    theta = np.arctan2(dy, dx)

    # 只在 r <= max_radius 的地方做 swirl
    mask = r <= max_radius

    # 正規化半徑 (0~1)
    r_normalized = np.zeros_like(r, dtype=np.float32)
    r_normalized[mask] = r[mask] / max_radius

    # 旋轉角度（距離越遠旋轉越小）
    rotation_angle = np.zeros_like(r, dtype=np.float32)
    rotation_angle[mask] = angle * (1.0 - r_normalized[mask])

    rotation_rad = np.deg2rad(rotation_angle)
    new_theta = theta + rotation_rad

    # 先預設為「不變形」
    x_new = x.copy()
    y_new = y.copy()

    # 只有在 mask 範圍內才改座標
    x_new[mask] = center_x + r[mask] * np.cos(new_theta[mask])
    y_new[mask] = center_y + r[mask] * np.sin(new_theta[mask])

    # 避免極少數浮點誤差跑出邊界
    x_new = np.clip(x_new, 0, width - 1)
    y_new = np.clip(y_new, 0, height - 1)

    map_x = x_new.astype(np.float32)
    map_y = y_new.astype(np.float32)

    # 用 REFLECT 當邊界模式，比 CONSTANT 好看很多
    result = cv2.remap(
        img,
        map_x,
        map_y,
        cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_REFLECT
    )

    return result


In [ ]:
from tqdm import tqdm

def batch_process_images(input_dir, output_base_dir, transformations):
    """
    批次處理圖片
    
    參數:
        input_dir: 輸入圖片目錄
        output_base_dir: 輸出基礎目錄
        transformations: 變形操作字典
    """
    # 確保輸入目錄存在
    if not os.path.exists(input_dir):
        print(f"錯誤：輸入目錄不存在: {input_dir}")
        return
    
    # 創建輸出基礎目錄
    os.makedirs(output_base_dir, exist_ok=True)
    
    # 為每個變形操作創建資料集目錄
    dataset_dirs = {}
    for dataset_name in transformations.keys():
        dataset_path = os.path.join(output_base_dir, dataset_name)
        os.makedirs(dataset_path, exist_ok=True)
        dataset_dirs[dataset_name] = dataset_path
        print(f"創建資料集目錄: {dataset_path}")
    
    # 獲取所有圖片文件
    image_extensions = ['.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.tif']
    image_files = []
    
    for file in os.listdir(input_dir):
        if any(file.lower().endswith(ext) for ext in image_extensions):
            image_files.append(file)
    
    print(f"\n找到 {len(image_files)} 張圖片")
    print(f"將生成 {len(image_files) * len(transformations)} 張變形圖片\n")
    
    # 統計資訊
    stats = {name: {'success': 0, 'failed': 0} for name in transformations.keys()}
    
    # 處理每張圖片
    for img_file in tqdm(image_files, desc="處理圖片"):
        img_path = os.path.join(input_dir, img_file)
        
        # 讀取圖片
        img = cv2.imread(img_path)
        if img is None:
            print(f"警告：無法讀取圖片 {img_file}")
            continue
        
        # 獲取文件名
        file_name_without_ext = os.path.splitext(img_file)[0]
        file_ext = os.path.splitext(img_file)[1]
        
        # 執行變形操作
        for dataset_name, transform_info in transformations.items():
            try:
                transform_func = transform_info['function']
                transform_params = transform_info.get('params', {})
                
                # 執行變形
                transformed_img = transform_func(img, **transform_params)
                
                # 保存圖片
                output_filename = f"{file_name_without_ext}{file_ext}"
                output_path = os.path.join(dataset_dirs[dataset_name], output_filename)
                cv2.imwrite(output_path, transformed_img)
                
                stats[dataset_name]['success'] += 1
                
            except Exception as e:
                print(f"錯誤：處理 {img_file} 的 {dataset_name} 變形時發生錯誤: {str(e)}")
                stats[dataset_name]['failed'] += 1
    
    return stats

In [ ]:
# 定義變形操作設定
transformations = {
    'swirl_180': {
        'function': swirl_effect,
        'params': {'angle': 30, 'strength': 0.5}
    }
}

# 設定輸入與輸出目錄
input_dir = ""
output_dir = ""

# 執行批次處理 (如果 input_dir 存在)
if os.path.exists(input_dir):
    print(f"開始處理目錄: {input_dir}")
    stats = batch_process_images(input_dir, output_dir, transformations)
    
    # 顯示結果
    print("\n處理統計:")
    for name, stat in stats.items():
        print(f"{name}: 成功 {stat['success']}, 失敗 {stat['failed']}")
else:
    print(f"找不到輸入目錄: {input_dir}，請修改 input_dir 變數")